In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from collections import defaultdict, Counter
from typing import List, Dict

from tqdm import tqdm
import polars as pl
import pandas as pd

In [4]:
train = pl.read_csv("/gdrive/MyDrive/amazon_kdd_2023/data/raw/sessions_train.csv")

# polarsでreadすると行数がずれる箇所があるため、pandasで読んだ後に変換する
test_pd = pd.read_csv("/gdrive/MyDrive/amazon_kdd_2023/data/raw/sessions_test_task3.csv")
test = pl.from_pandas(test_pd)

In [5]:
# prev_itemsの加工
def str2list(s):
    s = s.replace("[", "").replace("]", "").replace("'", "").replace("\n", " ").replace("\r", " ")
    s = s.split() # 空白で区切ってリスト化
    return s

train = train.with_columns(pl.col("prev_items").apply(str2list).alias("prev_items"))
test = test.with_columns(pl.col("prev_items").apply(str2list).alias("prev_items"))

In [6]:
# session_idの付与
train = train.with_columns(pl.Series(name="session_id", values=["train_" + str(i) for i in range(len(train))]))
test = test.with_columns(pl.Series(name="session_id", values=["test_" + str(i) for i in range(len(test))]))

In [7]:
train.head()

prev_items,next_item,locale,session_id
list[str],str,str,str
"[""B09W9FND7K"", ""B09JSPLN1M""]","""B09M7GY217""","""DE""","""train_0"""
"[""B076THCGSG"", ""B007MO8IME"", … ""B001B4TKA0""]","""B001B4THSA""","""DE""","""train_1"""
"[""B0B1LGXWDS"", ""B00AZYORS2"", … ""B00AZYORS2""]","""B0767DTG2Q""","""DE""","""train_2"""
"[""B09XMTWDVT"", ""B0B4MZZ8MB"", … ""B0B71CHT1L""]","""B0B4R9NN4B""","""DE""","""train_3"""
"[""B09Y5CSL3T"", ""B09Y5DPTXN"", ""B09FKD61R8""]","""B0BGVBKWGZ""","""DE""","""train_4"""


In [8]:
train.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task3/train_task3.parquet")

In [9]:
test.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task3/test_task3.parquet")